In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math

import pyro
import pyro.distributions as dist
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO

import torch
import torch.distributions.constraints as constraints

In [15]:
def model(data):
    # Define the hyperparameters of the prior knowledge
    alpha = torch.tensor(10.)
    beta = torch.tensor(10.)
    # Prior definition. Latent variable
    f = pyro.sample("f", dist.Beta(alpha, beta))

    # loop over the observed data
    for i in range(len(data)):
        
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])
    
def guide(data):
    alpha_q = pyro.param("alpha_q", torch.tensor(15.), constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.), constraint=constraints.positive)

    pyro.sample("f", dist.Beta(alpha_q, beta_q))

In [17]:
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

data = []
for _ in range(6):
    data.append(torch.tensor(1.0))
for _ in range(4):
    data.append(torch.tensor(0.0))


# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
# do gradient steps
n_steps = 5000
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')

# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

# here we use some facts about the Beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

..................................................